In [1]:
import requests
import scrapy

In [2]:
url = "http://goheels.com/roster.aspx?path=wvball"
resp = requests.get(url, headers={'User-Agent': 'UNC MEJO Class'},)

In [3]:
resp
#checks that the request is coming through

<Response [200]>

In [4]:
body_bytes = resp.content
# Body is in bytes... turn it in to a string by decoding it to UTF-8
body_str = body_bytes.decode('utf-8')

In [5]:
# Create a scrapy selector from the string content
# https://doc.scrapy.org/en/latest/topics/selectors.html
sel = scrapy.Selector(text=body_str)

In [6]:
table = sel.css('.sidearm-table')
table

[<Selector xpath="descendant-or-self::*[@class and contains(concat(' ', normalize-space(@class), ' '), ' sidearm-table ')]" data='<table class="sidearm-table sidearm-tabl'>,
 <Selector xpath="descendant-or-self::*[@class and contains(concat(' ', normalize-space(@class), ' '), ' sidearm-table ')]" data='<table class="sidearm-table sidearm-tabl'>]

In [7]:
# table.css('tr')
table.css('tr')[0].css('th').xpath('string()').extract() 

['#',
 'Full Name',
 'Pos.',
 'Ht.',
 'Year',
 'Hometown',
 'High School',
 'Previous School']

In [8]:
col_names = table.css('tr')[0].css('th').xpath('string()').extract()[0:3] 
col_names

['#', 'Full Name', 'Pos.']

In [9]:
other_rows = table.css('tr:not(:nth-child(0))')
# other_rows

In [10]:
by_number = {}
for row in other_rows:
    try:
        number = row.css('td').css('.roster_jerseynum').xpath('string()').extract()[0]
    except IndexError:
        continue
    # or year = row.css('td')[0].xpath('string()')[0].extract()

    number_data = []
    by_number[number] = number_data
by_number

{'1': [],
 '10': [],
 '12': [],
 '13': [],
 '14': [],
 '15': [],
 '16': [],
 '17': [],
 '18': [],
 '2': [],
 '25': [],
 '3': [],
 '4': [],
 '43': [],
 '5': [],
 '6': [],
 '7': [],
 '8': []}

In [11]:
by_number = {}
for row in other_rows:
    try:
        number = row.css('td').css('.roster_jerseynum').xpath('string()').extract()[0]
    except IndexError:
        continue
    # or year = row.css('td')[0].xpath('string()')[0].extract()

    # Loop through each data cell's links
    player = row.css('td:nth-child(2)')
    name = player.xpath('string()')[0].extract()
    name_href = player.css('a').xpath('@href').extract()[0]
    by_number[number] = {
        "number": number,
        "name": name,
        "href": name_href,
    }
by_number

{'1': {'href': '/roster.aspx?rp_id=8287',
  'name': 'Mariah Evans',
  'number': '1'},
 '10': {'href': '/roster.aspx?rp_id=8288',
  'name': 'Sydnye Fields',
  'number': '10'},
 '12': {'href': '/roster.aspx?rp_id=8298',
  'name': 'Julia Scoles',
  'number': '12'},
 '13': {'href': '/roster.aspx?rp_id=8289',
  'name': 'Mia Fradenburg',
  'number': '13'},
 '14': {'href': '/roster.aspx?rp_id=8295',
  'name': 'Madison Laufenberg',
  'number': '14'},
 '15': {'href': '/roster.aspx?rp_id=8293',
  'name': 'Keimaya Hunter',
  'number': '15'},
 '16': {'href': '/roster.aspx?rp_id=8285',
  'name': 'Raleigh Clark',
  'number': '16'},
 '17': {'href': '/roster.aspx?rp_id=8291',
  'name': 'Maddie Grace Hough',
  'number': '17'},
 '18': {'href': '/roster.aspx?rp_id=8292',
  'name': 'Sehrena Hull',
  'number': '18'},
 '2': {'href': '/roster.aspx?rp_id=12641',
  'name': 'Kendra Koetter',
  'number': '2'},
 '25': {'href': '/roster.aspx?rp_id=13855',
  'name': 'Greer Moseman',
  'number': '25'},
 '3': {'href'

In [12]:
def update_with_img(player):
#     print("updating")
#     print(roster)
    new_url = 'http://goheels.com/' + player["href"]
#     print("Calling GoHeels: {}".format(new_url))
    response = requests.get(new_url, headers={'User-Agent': 'UNC MEJO Class'},)
    content = response.content.decode('utf-8')
    sel = scrapy.Selector(text=content)
    img_div = sel.css('.sidearm-roster-player-image')
    img = img_div.css('img').xpath("@src").extract()[0]
    player["img_url"] = img
    
    bio_div = sel.css('#sidearm-roster-player-bio')
    bio = bio_div.xpath('string()').extract()[0]
    player["bio"] = bio
    
#     bio_div = sel.css('#sidearm-roster-player-bio')
#     bio = ''
#     for p in bio_div:
#         bio = bio.format(p)
#     player['bio'] = bio

for player in list(by_number.values()):
    update_with_img(player)

by_number

{'1': {'bio': '\r\n                        Biography\r\n                                                    \r\n                            Career Honors:\r\n» 2016 ACC Academic Honor Roll\r\n\r\n2017 (Sophomore):\xa0Suffered a concussion during the preseason that sidelined her for the entire fall\xa0• Requested a medical hardship waiver to redshirt the season.\r\n\r\n2016 (Freshman): Played in all 33 matches as a 6-2 setter with Abigail Curry, appearing in 122 of 125 sets • Totaled 503 assists, 168 digs, 12 aces, 12 kills and two blocks • Only made one attack error for a .478 hitting percentage • Ranked seventh all time among UNC freshmen in total assists and sixth in sets played • Led UNC’s offense to its second-highest hitting percentage in program history, an ACC-leading .273 clip • Registered three double-doubles, including back-to-back in the NCAA Tournament • Recorded double-digit assists in 27 of 33 appearances with seven 20-assist performances • Posted her best numbers of the 

In [13]:
import re

In [14]:
js_obj_rx = re.compile(r'.*?responsive-roster-bio\.ashx.*?({.*?})')

In [15]:
js_obj_rx

re.compile(r'.*?responsive-roster-bio\.ashx.*?({.*?})', re.UNICODE)

In [16]:
player_url = by_number["1"]["href"]
player_url = "http://goheels.com" + player_url 
player_url #unique url to player's page

'http://goheels.com/roster.aspx?rp_id=8287'

In [23]:
headers = {'User-Agent': 'UNC Journo Class'}
new_resp = requests.get(player_url, headers=headers)
body_bytes = new_resp.content
body_str = body_bytes.decode('utf-8')
body_str

'\r\n\r\n<!doctype html>\r\n<html id="ctl00_html" lang="en" class=" wide wvball sport wvball"> \r\n    <head><script>window.client_hostname = "goheels.com";window.server_name = "866411-LB16";window.client_division = "DI";window.targetting_criteria = {"page_template":"roster-player","sport_name":"wvball","sport_name_custom":"wvball","site":"unc","content_id":"roster-player-8287","sect":"wvolley"};window.client_shortname = "unc";window.dfp_network_code = "29658103";window.img_dfp_unit_name = "";window.block_dfp = false;window.reactions = null;</script><link rel="stylesheet" href="/less/sidearm-responsive-grid.less?_=636518877046678376"><link rel="stylesheet" href="https://s3.amazonaws.com/assets.sidearmsports.com/adaptive/css/normalize.min.css"><link rel="apple-touch-icon" href="/images/logos/site/site.png"><meta charset="utf-8"><meta name="viewport" content="width=device-width"><link rel="apple-touch-icon" href="/images/logos/site/site.png">\t\t\t\t\t\t\r\n\t\t\t<meta name="theme-color"

In [24]:
body_str

'\r\n\r\n<!doctype html>\r\n<html id="ctl00_html" lang="en" class=" wide wvball sport wvball"> \r\n    <head><script>window.client_hostname = "goheels.com";window.server_name = "866411-LB16";window.client_division = "DI";window.targetting_criteria = {"page_template":"roster-player","sport_name":"wvball","sport_name_custom":"wvball","site":"unc","content_id":"roster-player-8287","sect":"wvolley"};window.client_shortname = "unc";window.dfp_network_code = "29658103";window.img_dfp_unit_name = "";window.block_dfp = false;window.reactions = null;</script><link rel="stylesheet" href="/less/sidearm-responsive-grid.less?_=636518877046678376"><link rel="stylesheet" href="https://s3.amazonaws.com/assets.sidearmsports.com/adaptive/css/normalize.min.css"><link rel="apple-touch-icon" href="/images/logos/site/site.png"><meta charset="utf-8"><meta name="viewport" content="width=device-width"><link rel="apple-touch-icon" href="/images/logos/site/site.png">\t\t\t\t\t\t\r\n\t\t\t<meta name="theme-color"

In [25]:
parts = body_str.split('$.getJSON("/services/')[1:]
parts

['responsive-roster-bio.ashx", { type: \'stats\', rp_id: 8287, path: \'wvball\', year: 2017, player_id: 0 }, function(response) {\r\n                                self.isLoading(false);\r\n\t\t\t\t\t\t\t\tif (response)\r\n\t\t\t\t\t\t\t\t\tself.stats(response);\r\n                            } \r\n                        );\r\n                    };\r\n\r\n                    var Related = function() {\r\n                        var self = this;\r\n\r\n                        self.isLoading = ko.observable(true);\r\n                        self.stories = ko.observableArray([]);\r\n                        self.videos = ko.observableArray([]);\r\n                        self.photos = ko.observableArray([]);\r\n\r\n                        ',
 'responsive-roster-bio.ashx", { type: \'related\', rp_id: 8287, player_id: 3330 }, function(response) {\r\n                            self.isLoading(false);\r\n                            if (response.videos != null && response.videos.length > 0)\

In [26]:
# Now we run our regular expression on the joined 
# parts (put them back in to a single string)
# and capture the JS objects
captured = js_obj_rx.findall(''.join(parts))

In [27]:
# Note how similar the JS is to JSON...
captured

["{ type: 'stats', rp_id: 8287, path: 'wvball', year: 2017, player_id: 0 }",
 "{ type: 'related', rp_id: 8287, player_id: 3330 }"]

In [29]:
import json

In [30]:
# Turn it in to valid json by splitting everything apart
clean_objs = []
for obj_str in captured:
    # We only want the stats object...
    if 'stats' not in obj_str:
        continue

    # Remove the braces
    obj_str = obj_str.replace('{', '').replace('}', '')
    # Remove the quotes from whole string since some variables had them and some didn't, now will be able to put
        #proper quotes wherever they need to be in one fell swoop
    obj_str = obj_str.replace("'", '').replace('"', '')
    # Split apart on commas
    obj_pairs = obj_str.split(',')
    # Now it looks like [" type: related", " rp_id: 9450", ...]
    # So we need to wrap everything in quotes to make it json so lets get separate values
    obj_pairs = [x.split(":") for x in obj_pairs]
    # Now it looks like [(' type', ' related'), ]
    # and we need to clean up all the leading / trailing white space
    # and wrap each value in double quotes
    print(obj_pairs)
    clean_pairs = []
    for pair in obj_pairs:
        clean_pairs.append(['"{}"'.format(p.strip()) for p in pair]) #format() strips out extra spacing
    print(clean_pairs)
    # And join them together with colons
    colonized = [":".join(p) for p in clean_pairs]
    print(colonized)
    # So now colonized is ['"type":"stats"']
    # and we need to join those together with commas
    commas = ','.join(colonized)
    print(commas)
    # And make it look like a JSON object...
    json_str = "{" + commas + "}"
    print(json_str)
    # And make it a python dictionary
    clean_objs.append(json.loads(json_str))

[[' type', ' stats'], [' rp_id', ' 8287'], [' path', ' wvball'], [' year', ' 2017'], [' player_id', ' 0 ']]
[['"type"', '"stats"'], ['"rp_id"', '"8287"'], ['"path"', '"wvball"'], ['"year"', '"2017"'], ['"player_id"', '"0"']]
['"type":"stats"', '"rp_id":"8287"', '"path":"wvball"', '"year":"2017"', '"player_id":"0"']
"type":"stats","rp_id":"8287","path":"wvball","year":"2017","player_id":"0"
{"type":"stats","rp_id":"8287","path":"wvball","year":"2017","player_id":"0"}


In [31]:
clean_objs

[{'path': 'wvball',
  'player_id': '0',
  'rp_id': '8287',
  'type': 'stats',
  'year': '2017'}]

In [32]:
# That was a lot of work! but now we can go get stats directly
#double splat
#keyword arg
stats_url = (
    "http://goheels.com/services/responsive-roster-bio.ashx?"
    "type={type}&rp_id={rp_id}&path={path}&year={year}"
    "&player_id={player_id}"
).format(**clean_objs[0]) #anything in curly brackets and a name, it will assume the obj you're calling has a name
                            #like that. automatically fills in variables from the obj

In [33]:
stats_url

'http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=8287&path=wvball&year=2017&player_id=0'

In [34]:
stats_resp = requests.get(stats_url, headers=headers)
stats_sel = scrapy.Selector(text=stats_resp.content.decode('utf-8'))

In [35]:
stats_resp.content

b'{"current_stats":"","career_stats":"\\r\\n\\r\\n    <section>\\r\\n        <h5>Offensive Statistics</h5>\\r\\n        \\r\\n        <div class=\\"sidearm-table-overflow-on-x-large\\">\\r\\n            <table class=\\"sidearm-table highlight-column-hover\\">\\r\\n                <caption class=\\"hide\\">Career Offensive Statistics</caption>\\r\\n                <thead>\\r\\n                    <tr>\\r\\n                        <th scope=\\"col\\" class=\\"text-hide\\">Season</th>\\r\\n                        <th scope=\\"col\\">SP</th>\\r\\n                        <th scope=\\"col\\">MP</th>\\r\\n                        <th scope=\\"col\\">K</th>\\r\\n                        <th scope=\\"col\\">K/S</th>\\r\\n                        <th scope=\\"col\\">E</th>\\r\\n                        <th scope=\\"col\\">TA</th>\\r\\n                        <th scope=\\"col\\">%</th>\\r\\n                        <th scope=\\"col\\">A</th>\\r\\n                        <th scope=\\"col\\">A/S</th>\\r

In [41]:
stats_html = json.loads(stats_resp.content.decode("utf-8"))["career_stats"]

In [42]:
html_sel = scrapy.Selector(text=stats_html)

In [44]:
html_sel

<Selector xpath=None data='<html><body><section>\r\n        <h5>Offen'>

In [58]:
thead = html_sel.css('thead')[0]
thead

<Selector xpath='descendant-or-self::thead' data='<thead>\r\n                    <tr>\r\n     '>

In [74]:
# Get all the column names from the first row's table heading (th) elements
col_names = thead.css('tr:nth-child(1) th')[1:].xpath('string()').extract()
col_names

['SP', 'MP', 'K', 'K/S', 'E', 'TA', '%', 'A', 'A/S', 'SA', 'SA/S', 'SE']

In [75]:
tbody = html_sel.css('tbody')[0]

In [76]:
col_content = tbody.css('tr:first-of-type:nth-child(1) td').xpath('string()').extract()
col_content

['122',
 '33',
 '12',
 '0.10',
 '1',
 '23',
 '.478',
 '503',
 '4.12',
 '12',
 '0.10',
 '26']

In [77]:
col_names_content = list(zip(col_names, col_content))
col_names_content

[('SP', '122'),
 ('MP', '33'),
 ('K', '12'),
 ('K/S', '0.10'),
 ('E', '1'),
 ('TA', '23'),
 ('%', '.478'),
 ('A', '503'),
 ('A/S', '4.12'),
 ('SA', '12'),
 ('SA/S', '0.10'),
 ('SE', '26')]

In [80]:
year_rows = html_sel.css('tr th:first-of-type').xpath('string()').extract()
year_rows

['Season', '2016', 'Total', 'Season', '2016', 'Total']

In [ ]:

for row in other_rows:
    try:
        year = row.css('td')[0].xpath('string()').extract()[0]
    except IndexError:
        continue
        
    year_data = []
    by_year[year] = year_data